In [1]:
# Defining a function to wite(.csv) a file to the user location.
def writeDF(dataframe,filename,dir_path):
    dataframe.to_csv(path_or_buf = outDir+filename+'.csv')
    size = str(round((os.path.getsize(outDir+filename+'.csv')/1000),1))
    print('File: '+filename+'.csv created at '+outDir+ '\nSize of file :'+size+ ' KB' )
    
# Defining a method to wite(.xlsx) a file to the user location.
def writeDF_xl(dataframe,filename,dir_path):
    writer = pd.ExcelWriter(outDir+filename+'.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
    dataframe.to_excel(writer,sheet_name= filename)
    writer.close()
    size = str(round((os.path.getsize(outDir+filename+'.xlsx')/1000),1))
    print('File: '+filename+'.xlsx created at '+outDir+ '\nSize of file :'+size+ ' KB' )

In [35]:
# Creating a custom function to wite(.csv) a file to the current working directory.
def writeDF(dataframe,outFileName):
    '''
    dataframe : Dataframe name
    Pass the dataframe name & outFileName, this function will create a file 'outFileName.csv' at current working directory.
    '''
    dataframe.to_csv(path_or_buf = './'+outFileName+'.csv',index= False)
    size = str(round((os.path.getsize('./'+outFileName+'.csv')/1000),1))
    print('File:'+ outFileName+'.csv created at current working directory \nSize of file :\t'+size+ ' KB' )

In [45]:
def impute_numeric_values(dataframe,column,approach):
    '''impute_values function will impute the missing data(numeric) in the dataframe column.
        dataframe :: Dataframe Name
        column    :: Column name(numeric)
        approach  :: 1,2,3 (default value 1)
            1. Delete the row
            2. Replace missing value with mean value.
            3. Replace missing value with 0
    '''
    if approach == 1:
        dataframe_filtered = dataframe[column].dropna()
    if approach == 2:
        dataframe_filtered = dataframe[column].fillna(dataframe[column].mean())
    if approach == 3:
        dataframe_filtered = dataframe[column].fillna(0)        
        
    print('\nNumber of Records (after imputing missing values): ',dataframe_filtered.size)
    return dataframe_filtered

In [61]:
# Load Data
import pandas as pd
uber = pd.read_csv('C:/Users/ankit.bhatia/Documents/Cognos/uber_coordinates.csv',sep='|',header=None,\
                  names = ["Latitude", "Longitude", "Base", "Timestamp","Year","Quarter","Month","Day","DayName","Cluster"])
uber.head()

,Latitude,Longitude,Base,Timestamp,Year,Quarter,Month,Day,DayName,Cluster
0,40.7293,-73.9920,B02512,2014-06-01 00:00:00.0,2014,6,2,22,Sunday,0
1,40.7131,-74.0097,B02512,2014-06-01 00:01:00.0,2014,6,2,22,Sunday,0
2,40.3461,-74.6610,B02512,2014-06-01 00:04:00.0,2014,6,2,22,Sunday,2
3,40.7555,-73.9833,B02512,2014-06-01 00:04:00.0,2014,6,2,22,Sunday,5
4,40.6880,-74.1831,B02512,2014-06-01 00:07:00.0,2014,6,2,22,Sunday,2


In [48]:
uber.drop('Date/Time',inplace=True,axis=1)

uber.head()


,Lat,Lon,Base,TIMESTAMP,Year,Month,Quarter,Week,Day
0,40.7293,-73.9920,B02512,2014-06-01 00:00:00,2014,6,2,22,Sunday
1,40.7131,-74.0097,B02512,2014-06-01 00:01:00,2014,6,2,22,Sunday
2,40.3461,-74.6610,B02512,2014-06-01 00:04:00,2014,6,2,22,Sunday
3,40.7555,-73.9833,B02512,2014-06-01 00:04:00,2014,6,2,22,Sunday
4,40.6880,-74.1831,B02512,2014-06-01 00:07:00,2014,6,2,22,Sunday


In [62]:
import os
writeDF(uber,'uber_coordinates_sep')

File:uber_coordinates_sep.csv created at current working directory 
Size of file :	45903.7 KB


In [54]:
def crosstab(df,col):
    '''
    df : Dataframe
    col: Column Name
    '''
    crosstab = pd.crosstab(df[col], df['loan_status'],margins=True)
    crosstab['Probability_Charged Off'] = round((crosstab['Charged Off']/crosstab['All']),3)
    crosstab = crosstab[0:-1]
    return crosstab

In [52]:
# Probability of charge off
def bivariate_prob(df,col,stacked= True):
    '''
    df      : Dataframe
    col     : Column Name
    stacked : True(default) for Stacked Bar
    '''
    # get dataframe from crosstab function
    plotCrosstab = crosstab(df,col)
    
    linePlot = plotCrosstab[['Probability_Charged Off']]      
    barPlot =  plotCrosstab.iloc[:,0:2]
    ax = linePlot.plot(figsize=(20,8), marker='o',color = 'b')
    ax2 = barPlot.plot(kind='bar',ax = ax,rot=1,secondary_y=True,stacked=stacked)
    ax.set_title(df[col].name.title()+' vs Probability Charge Off',fontsize=20,weight="bold")
    ax.set_xlabel(df[col].name.title(),fontsize=14)
    ax.set_ylabel('Probability of Charged off',color = 'b',fontsize=14)
    ax2.set_ylabel('Number of Applicants',color = 'g',fontsize=14)
    plt.show()

In [53]:
def bivariate_prob2(df,col,norm='m'):
    crosstab = pd.crosstab(df[col], df['loan_status'],margins=True)
    crosstab['Probability_Charged Off'] = round((crosstab['Charged Off']/crosstab['All']),3)
    #normalize using z-scale
    if norm == 'z':
        crosstab=(crosstab-crosstab.mean())/(crosstab.std())
    #normalize using min-max scale
    else:
        crosstab=(crosstab-crosstab.min())/(crosstab.max()-crosstab.min())

    cross = (crosstab.loc[:,['Fully Paid','Charged Off']])[0:-1]
    cross_prob = (crosstab.loc[:,['Probability_Charged Off']])[0:-1]
    ax = cross_prob.plot(figsize=(20,8), marker='o',color = ['b'])
    cross.plot(kind='bar',ax = ax,rot=1)
    plt.show()